In [1]:
%load_ext autoreload
%autoreload 2

# Downloading data from sRNATarBase

http://aibcenter.com/srnatarbase/index.php?r=srnaTarget/index

# Imports

In [2]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [11]:
def scrape_rna_sequences(url):
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the page content with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find elements containing RNA sequences
            seqs = {}
            seqs['sRNA'] = soup.find('td', string='sRNA').nextSibling.text.replace('\n', '')
            seqs['Target'] = soup.find('td', string='Target').nextSibling.text.replace('\n', '')
            seqs['sRNA Sequence'] = soup.find('td', string='sRNA Seq').nextSibling.text.replace('\n', '')
            seqs['Target Sequence'] = soup.find('td', string='Target Seq').nextSibling.text.replace('\n', '')
            return pd.DataFrame.from_dict({k: [v] for k, v in seqs.items()})
        elif response.status_code == 500:
            return 1
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during the request: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# URL of the page containing RNA sequences
url = "http://aibcenter.com/srnatarbase/index.php?r=srnaTarget/index"

url = "http://aibcenter.com/srnatarbase/index.php?r=srnaTarget/view&id=17"

# Call the function to scrape RNA sequences
seqs = scrape_rna_sequences(url)


In [13]:
url = "http://aibcenter.com/srnatarbase/index.php?r=srnaTarget/view&id="
i = 1
df = pd.DataFrame()
while seqs is not None:
    seqs = scrape_rna_sequences(url + str(i))
    i += 1
    if (seqs is not None) and (type(seqs) != int):
        df = pd.concat([df, seqs])
    
    

Failed to retrieve data. Status code: 404


In [14]:
df.reset_index()

,index,sRNA,Target,sRNA Sequence,Target Sequence
0,0,dsrA [chromosome:NC_000913.3] [Gene ID:946470],hns [chromosome:NC_000913.3] [Gene ID:945829],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGAGCGAAGCACTTAAAATTCTGAACAACATCCGTACTCTTCGTG...
1,0,dsrA [chromosome:NC_000913.3] [Gene ID:946470],rbsD [chromosome:NC_000913.3] [Gene ID:948267],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGAAAAAAGGCACCGTTCTTAATTCTGATATTTCATCGGTGATCT...
2,0,dsrA [chromosome:NC_000913.3] [Gene ID:946470],argR [chromosome:NC_000913.3] [Gene ID:947861],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGCGAAGCTCGGCTAAGCAAGAAGAACTAGTTAAAGCATTTAAAG...
3,0,dsrA [chromosome:NC_000913.3] [Gene ID:946470],ilvI [chromosome:NC_000913.3] [Gene ID:948793],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATGGAGATGTTGTCTGGAGCCGAGATGGTCGTCCGATCGCTTATCG...
4,0,ChiX [chromosome:NC_003197.1] [Gene ID:],ybfM [chromosome:NC_003197.1] [Gene ID:1252207],GATCCGAAGCGAAAGCGTCGGGATAATAATAACGATGAAATTCCTC...,ATGCGTACGTTTAGTGGCAAACGTAGTACGCTGGCTCTGGCTATCG...
...,...,...,...,...,...
764,0,RnaC [chromosome:NC_000964.3] [Gene ID:],abrB [chromosome:NC_000964.3] [Gene ID:937009],,ATGTTTATGAAATCTACTGGTATTGTACGTAAAGTTGATGAATTAG...
765,0,VrrA [chromosome:] [Gene ID:],vrp [chromosome:] [Gene ID:],,
766,0,SorY [chromosome:NC_007493.2] [Gene ID:],takP [chromosome:NC_007493.2] [Gene ID:],,ATGGATCGTCGTTCATTCATCACCAAGGCCGCCGTGGGAGGGGCCG...
767,0,BSR0602 [chromosome:NC_003317.1] [Gene ID:],gntR [chromosome:NC_003317.1] [Gene ID:],,ATGGTAAAAGAGCCACGTTTTAGCAGTCGTCTGGAAAAGCCCCTCA...
